In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os


def sift():
    # getting grayscaled image
    root = os.getcwd()
    imgPath = os.path.join(root, 'img.jpg')
    imgGray = cv.imread(imgPath, cv.IMREAD_GRAYSCALE)

    # getting sift descriptors
    sift = cv.SIFT_create()
    keypoints = sift.detect(imgGray, None) 
    imgGray = cv.drawKeypoints(
        imgGray, keypoints, imgGray, 
        flags=cv.DrawMatchesFlags_DRAW_RICH_KEYPOINTS # rich keypoints will show orientations and etc.. desciptors
    )

    plt.figure()
    plt.imshow(imgGray)
    plt.show()
    
if __name__ == 'main':
    sift()